In [1]:
#Non-Event = 0, Event = 1 & Super-Event = 2
#No Super Event for Saffola Oils & Oats

In [2]:
import pandas as pd
import numpy as np
from pulp import *
import warnings
warnings.filterwarnings("ignore")
    
sim_input=pd.read_excel("Optimization_Input_Saffola_oil.xlsx")
#model_output['SPEND'] = model_output['SPEND']/6
sim_input['VALUE'] = sim_input['SPEND']*sim_input['ROI']
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
sim_input.head(10)

,FEATURE,SPEND,Event_tag,ROI,VALUE
0,Facebook Spend,2.784327,1,2.70,7.517682
1,Instagram Spend,0.416003,1,0.00,0.000000
2,G Adwords Spend,4.427988,1,0.44,1.948315
3,YouTube Spend,4.746917,1,2.30,10.917908
4,Display Spend,0.000143,1,0.00,0.000000
5,Facebook Collab Spend,3.077647,1,4.72,14.526492
6,Instagram Collab Spend,0.366750,1,0.00,0.000000
7,AMS Display Spend,5.666667,1,0.00,0.000000
8,AMS Brand Search Spend,1.028545,1,5.00,5.142727
9,AMS Product Search Spend,3.428912,1,5.00,17.144562


In [4]:
format1 = {'Format':['Youtube Spend', 'Google Display Spend', 'G Adwords Spend', 'Facebook Spend', 'Instagram Spend', np.NaN, np.NaN, 'AMS Search Display', 'AMS Search Product', 'AMS Search Brand',
                 'AMS Display', np.NaN, np.NaN, np.NaN, np.NaN, 'JBP Spends', np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN,
                 np.NaN, np.NaN, np.NaN, np.NaN, 'Amazon', 'Flipkart', 'Big Basket']}

format1 =pd.DataFrame(format1)

In [5]:
sim_input.head()

,FEATURE,SPEND,Event_tag,ROI,VALUE
0,Facebook Spend,2.784327,1,2.70,7.517682
1,Instagram Spend,0.416003,1,0.00,0.000000
2,G Adwords Spend,4.427988,1,0.44,1.948315
3,YouTube Spend,4.746917,1,2.30,10.917908
4,Display Spend,0.000143,1,0.00,0.000000


## gradual reduction for min threshold- 13/07/2021

In [6]:
def liner_programming_scenario(data, change_value =1, event_tag =0):
    
    sim_input = data.copy()
    temp_df =sim_input[['FEATURE', 'ROI']].drop_duplicates()
    temp_df['spend_type']=''

#     model_output['spend_type'][model_output.SPEND>=4]='hs'

    temp_df['spend_type'][sim_input.ROI==0]='ns'

#     model_output=model_output.sort_values(by=['SPEND'],ascending=False).reset_index(drop=True)
#     print(model_output)

    temp_df['upper_range']=0
    temp_df['lower_range']=0 

    #model_output['idx']=[str(i+1)+model_output.spend_type[i] for i in model_output.index]

    temp_df['upper_range'][temp_df.spend_type.str.contains('ns')]=0
    temp_df['lower_range'][temp_df.spend_type.str.contains('ns')]=0
   ###### Non event
    if event_tag ==0:
        
        Total_spend =  sim_input[sim_input['Event_tag'] == 0]['SPEND'].sum()
        Total_value = sim_input[sim_input['Event_tag'] == 0]['VALUE'].sum()
        
        #### Decrease scenario
        if 0<=float(change_value) <1 :
            temp_df['upper_range'][temp_df.FEATURE.str.contains('YouTube')]=48.2
            temp_df['lower_range'][temp_df.FEATURE.str.contains('YouTube')]=13.4*change_value
            
            temp_df['upper_range'][temp_df.FEATURE.str.contains('G Adwords')]=7.2
            temp_df['lower_range'][temp_df.FEATURE.str.contains('G Adwords')]=1.4*change_value
            
            temp_df['upper_range'][temp_df.FEATURE.str.contains('Facebook')]=10.6
            temp_df['lower_range'][temp_df.FEATURE.str.contains('Facebook')]=2.4*change_value
                      
            temp_df['upper_range'][temp_df.FEATURE.str.contains('Facebook Collab')]=7.0
            temp_df['lower_range'][temp_df.FEATURE.str.contains('Facebook Collab')]=0.0*change_value
            
            temp_df['upper_range'][temp_df.FEATURE.str.contains('AMS Product Search')]=2.77
            temp_df['lower_range'][temp_df.FEATURE.str.contains('AMS Product Search')]=2.36*change_value
            
            temp_df['upper_range'][temp_df.FEATURE.str.contains('AMS Brand Search')]=0.42
            temp_df['lower_range'][temp_df.FEATURE.str.contains('AMS Brand Search')]=0.31*change_value
            
            
            
            temp_df['upper_range'][temp_df.FEATURE.str.contains('JBP Spends (Amazon)')]=10.13
            temp_df['lower_range'][temp_df.FEATURE.str.contains('JBP Spends (Amazon)')]=7.75*change_value
            temp_df['upper_range'][temp_df.FEATURE.str.contains('JBP Spends (Flipkart)')]=16.69
            temp_df['lower_range'][temp_df.FEATURE.str.contains('JBP Spends (Flipkart)')]=9.65*change_value
            temp_df['upper_range'][temp_df.FEATURE.str.contains('JBP Spends (Big Basket)')]=19.87
            temp_df['lower_range'][temp_df.FEATURE.str.contains('JBP Spends (Big Basket)')]=13.48*change_value
            
        else:
            temp_df['upper_range'][temp_df.FEATURE.str.contains('YouTube')]=48.2
            temp_df['lower_range'][temp_df.FEATURE.str.contains('YouTube')]=13.4
            
            temp_df['upper_range'][temp_df.FEATURE.str.contains('G Adwords')]=7.2
            temp_df['lower_range'][temp_df.FEATURE.str.contains('G Adwords')]=1.4
            
            temp_df['upper_range'][temp_df.FEATURE.str.contains('Facebook')]=10.6
            temp_df['lower_range'][temp_df.FEATURE.str.contains('Facebook')]=2.4
                      
            temp_df['upper_range'][temp_df.FEATURE.str.contains('Facebook Collab')]=7.0
            temp_df['lower_range'][temp_df.FEATURE.str.contains('Facebook Collab')]=0.0
            
            temp_df['upper_range'][temp_df.FEATURE.str.contains('AMS Product Search')]=2.77
            temp_df['lower_range'][temp_df.FEATURE.str.contains('AMS Product Search')]=2.36
            
            temp_df['upper_range'][temp_df.FEATURE.str.contains('AMS Brand Search')]=0.42
            temp_df['lower_range'][temp_df.FEATURE.str.contains('AMS Brand Search')]=0.31
            
            
            
            temp_df['upper_range'][temp_df.FEATURE.str.contains('JBP Spends (Amazon)')]=10.13
            temp_df['lower_range'][temp_df.FEATURE.str.contains('JBP Spends (Amazon)')]=7.75
            temp_df['upper_range'][temp_df.FEATURE.str.contains('JBP Spends (Flipkart)')]=16.69
            temp_df['lower_range'][temp_df.FEATURE.str.contains('JBP Spends (Flipkart)')]=9.65
            temp_df['upper_range'][temp_df.FEATURE.str.contains('JBP Spends (Big Basket)')]=19.87
            temp_df['lower_range'][temp_df.FEATURE.str.contains('JBP Spends (Big Basket)')]=13.48
            
        
        
        lower_range_tot_spend = temp_df['lower_range'].sum()
        print("Lower range")
        print(lower_range_tot_spend)
        upper_range_tot_spend = temp_df['upper_range'].sum()
        print("Upper")
        print(upper_range_tot_spend)
        
        
        ###################################################################################
    else: 
        
            if event_tag == 1:
                Total_spend =  sim_input[sim_input['Event_tag'] == 1]['SPEND'].sum()
                Total_value = sim_input[sim_input['Event_tag'] == 1]['VALUE'].sum()
                print("Priniting Total")
                print(Total_spend)
                print(Total_value)
                lower_range_tot_spend = 20
#                 if float(change_value) <=1.10:
                upper_range_tot_spend = 75
#                 else:
#                     upper_range_tot_spend = 77
                    
            else:
                Total_spend =  sim_input[sim_input['Event_tag'] == 2]['SPEND'].sum()
                Total_value = sim_input[sim_input['Event_tag'] == 2]['VALUE'].sum()
                lower_range_tot_spend = 43.6
                if float(change_value) <=1.10:
                    upper_range_tot_spend = 66
                else:
                    upper_range_tot_spend = 69
                    
            if 0<=float(change_value) <1:
                
                temp_df['upper_range'][temp_df.FEATURE.str.contains('YouTube')]=18.67
                temp_df['lower_range'][temp_df.FEATURE.str.contains('YouTube')]=7.12*change_value
            
                temp_df['upper_range'][temp_df.FEATURE.str.contains('G Adwords')]=9.11
                temp_df['lower_range'][temp_df.FEATURE.str.contains('G Adwords')]=0.25*change_value
            
                temp_df['upper_range'][temp_df.FEATURE.str.contains('Facebook')]=8.35
                temp_df['lower_range'][temp_df.FEATURE.str.contains('Facebook')]=2.02*change_value
                      
                temp_df['upper_range'][temp_df.FEATURE.str.contains('Facebook Collab')]=9.23
                temp_df['lower_range'][temp_df.FEATURE.str.contains('Facebook Collab')]=0.0*change_value
                
                temp_df['upper_range'][temp_df.FEATURE.str.contains('AMS Product Search')]=4.75
                temp_df['lower_range'][temp_df.FEATURE.str.contains('AMS Product Search')]=2.77*change_value
            
                temp_df['upper_range'][temp_df.FEATURE.str.contains('AMS Brand Search')]=1.13
                temp_df['lower_range'][temp_df.FEATURE.str.contains('AMS Brand Search')]=0.09*change_value
                
                
                
                temp_df['upper_range'][temp_df.FEATURE.str.contains('JBP Spends (Amazon)')]=11.47
                temp_df['lower_range'][temp_df.FEATURE.str.contains('JBP Spends (Amazon)')]=8.92*change_value
                temp_df['upper_range'][temp_df.FEATURE.str.contains('JBP Spends (Flipkart)')]=11.32
                temp_df['lower_range'][temp_df.FEATURE.str.contains('JBP Spends (Flipkart)')]=7.10*change_value
                temp_df['upper_range'][temp_df.FEATURE.str.contains('JBP Spends (Big Basket)')]=21.29
                temp_df['lower_range'][temp_df.FEATURE.str.contains('JBP Spends (Big Basket)')]=19.05*change_value
             
                 
            else:
                temp_df['upper_range'][temp_df.FEATURE.str.contains('YouTube')]=18.67
                temp_df['lower_range'][temp_df.FEATURE.str.contains('YouTube')]=7.12
            
                temp_df['upper_range'][temp_df.FEATURE.str.contains('G Adwords')]=9.11
                temp_df['lower_range'][temp_df.FEATURE.str.contains('G Adwords')]=0.25
            
                temp_df['upper_range'][temp_df.FEATURE.str.contains('Facebook')]=8.35
                temp_df['lower_range'][temp_df.FEATURE.str.contains('Facebook')]=2.02
                      
                temp_df['upper_range'][temp_df.FEATURE.str.contains('Facebook Collab')]=9.23
                temp_df['lower_range'][temp_df.FEATURE.str.contains('Facebook Collab')]=0.0
                
                temp_df['upper_range'][temp_df.FEATURE.str.contains('AMS Product Search')]=4.75
                temp_df['lower_range'][temp_df.FEATURE.str.contains('AMS Product Search')]=2.77
            
                temp_df['upper_range'][temp_df.FEATURE.str.contains('AMS Brand Search')]=1.13
                temp_df['lower_range'][temp_df.FEATURE.str.contains('AMS Brand Search')]=0.09
                
                
                
                temp_df['upper_range'][temp_df.FEATURE.str.contains('JBP Spends (Amazon)')]=11.47
                temp_df['lower_range'][temp_df.FEATURE.str.contains('JBP Spends (Amazon)')]=8.92
                temp_df['upper_range'][temp_df.FEATURE.str.contains('JBP Spends (Flipkart)')]=11.32
                temp_df['lower_range'][temp_df.FEATURE.str.contains('JBP Spends (Flipkart)')]=7.10
                temp_df['upper_range'][temp_df.FEATURE.str.contains('JBP Spends (Big Basket)')]=21.29
                temp_df['lower_range'][temp_df.FEATURE.str.contains('JBP Spends (Big Basket)')]=19.05
                
                 
                

        
    spend_data= temp_df.copy()
    print("#################### Spend data ############################")
    print(spend_data)
    prob=LpProblem("Spend Constant Scenario Livon",LpMaximize)


    #creating LP variables
    spend_variables = LpVariable.dicts("spend_variable",[i for i in spend_data.FEATURE] ,lowBound=0,cat='Continuous')
    print("Spend Variables")
    print(spend_variables)

    #  forming maxima function using model output ROI and feature
    model_coeff = dict(zip(spend_data.FEATURE,spend_data.ROI))
    prob += lpSum([model_coeff[i]*spend_variables[i] for i in spend_data.FEATURE])

#     Total_spend = 49.16082729075
     
    #Value = ROI_fb*Facebook+ ROI_insta*Insta_spend  
        
    prob += lpSum([spend_variables[i] for i in spend_data.FEATURE]) >= lower_range_tot_spend 
    prob += lpSum([spend_variables[i] for i in spend_data.FEATURE]) <= upper_range_tot_spend
        
    prob += lpSum([spend_variables[i] for i in spend_data.FEATURE]) >= change_value*0.99*Total_spend
    prob += lpSum([spend_variables[i] for i in spend_data.FEATURE]) <=change_value*1.01*Total_spend

    # Forming LP equations with constraints
    for i,row in spend_data.iterrows():
    #         print(row['spend_type'])
        prob+=lpSum(spend_variables[row["FEATURE"]])>=row['lower_range']
        prob+=lpSum(spend_variables[row["FEATURE"]])<=row['upper_range']
    #solving LP 
    print(prob)
    prob.solve()

    print("Status:", LpStatus[prob.status])
    #     print(prob)

    #getting Spend
    new_spend={}
    variables=dict([(str(value), key) for key, value in spend_variables.items()])
    print(variables)
    for v in prob.variables():
        if v.varValue>0:
            print(v.name[15:], "=", v.varValue)
#             print(v.name)
#             print(variables[v.name])
            new_spend[variables[v.name]]=v.varValue

    
    #Analysing Output
    spend_data["new_spend"]=0.0
    

    for i,v in new_spend.items():
        spend_data["new_spend"][spend_data.FEATURE==i]=v
        
    

    spend_data["new_bpm"]=spend_data.new_spend*spend_data.ROI
    
    print('---------------Overview-------------------')
    print("EARLIER SPEND: {}".format(Total_spend))
    print("NEW SPEND: {} increased/decreased by {} %".format(spend_data['new_spend'].sum(),round((spend_data['new_spend'].sum()-Total_spend)/Total_spend,4)*100))
    print('EXISTING VALUE: {}'.format(Total_value))
    print('NEW VALUE: {} increased/decreased by {}%'.format(spend_data.new_bpm.sum(),round((spend_data.new_bpm.sum()-Total_value)/Total_value,4)*100))
    # spend_data.replace({'FEATURE' : { 'GOOGLE_DISPLAY_SPEND' : 'Display', 'AMS SEARCH SPEND' : 'AMS Search Product', 'JBP_SPEND' : 'JBP Spends', 
    #                      'YOUTUBE_SPEND': 'YT', 'INSTAGRAM_SPEND': 'IG', 'FACEBOOK_SPEND': 'FB', 'GADS_SPEND': 'G adwords',  'AMS_SPON_DISPLAY_SPEND': 'AMS Search Display'}}, inplace = True)
    spend_data_new_spend = spend_data[['FEATURE', 'new_spend']]
    spend_data_new_spend['new_spend'] = spend_data_new_spend['new_spend']
    
    
    
    return(spend_data_new_spend.copy())


In [7]:
b= liner_programming_scenario(sim_input, change_value =1.1, event_tag = 0)

Lower range
19.869999999999997
Upper
76.19
#################### Spend data ############################
                     FEATURE   ROI spend_type  upper_range  lower_range
0             Facebook Spend  2.70                   10.60         2.40
1            Instagram Spend  0.00         ns         0.00         0.00
2            G Adwords Spend  0.44                    7.20         1.40
3              YouTube Spend  2.30                   48.20        13.40
4              Display Spend  0.00         ns         0.00         0.00
5      Facebook Collab Spend  4.72                    7.00         0.00
6     Instagram Collab Spend  0.00         ns         0.00         0.00
7          AMS Display Spend  0.00         ns         0.00         0.00
8     AMS Brand Search Spend  5.00                    0.42         0.31
9   AMS Product Search Spend  5.00                    2.77         2.36
10  AMS Search Display Spend  0.00         ns         0.00         0.00
Spend Variables
{'Facebook Spend

In [8]:
print(sim_input.FEATURE.unique())

['Facebook Spend' 'Instagram Spend' 'G Adwords Spend' 'YouTube Spend'
 'Display Spend' 'Facebook Collab Spend' 'Instagram Collab Spend'
 'AMS Display Spend' 'AMS Brand Search Spend' 'AMS Product Search Spend'
 'AMS Search Display Spend']


In [9]:
##### MAX BPM Event 
Output_event = pd.DataFrame()
outputs_event = pd.DataFrame(sim_input.FEATURE.unique())
outputs_event.columns = ['FEATURE']
for i in np.arange(0, 2.05, 0.05):
    print("############ change ##########")
    print(i)
    b= liner_programming_scenario(sim_input, change_value = i, event_tag = 1)
#     col_name = round(i)
    b.rename(columns = {'new_spend': i}, inplace = True)
    b.drop('FEATURE', axis =1, inplace = True)
    print(b)
    outputs_event= pd.concat([outputs_event, b], axis =1)

############ change ##########
0.0
Priniting Total
25.94400883333333
57.19768650666667
#################### Spend data ############################
                     FEATURE   ROI spend_type  upper_range  lower_range
0             Facebook Spend  2.70                    8.35            0
1            Instagram Spend  0.00         ns         0.00            0
2            G Adwords Spend  0.44                    9.11            0
3              YouTube Spend  2.30                   18.67            0
4              Display Spend  0.00         ns         0.00            0
5      Facebook Collab Spend  4.72                    9.23            0
6     Instagram Collab Spend  0.00         ns         0.00            0
7          AMS Display Spend  0.00         ns         0.00            0
8     AMS Brand Search Spend  5.00                    1.13            0
9   AMS Product Search Spend  5.00                    4.75            0
10  AMS Search Display Spend  0.00         ns         0.00  

In [10]:
outputs_event

,FEATURE,0.0,0.05,0.1,0.15000000000000002,0.2,0.25,0.30000000000000004,0.35000000000000003,0.4,0.45,0.5,0.55,0.6000000000000001,0.65,0.7000000000000001,0.75,0.8,0.8500000000000001,0.9,0.9500000000000001,1.0,1.05,1.1,1.1500000000000001,1.2000000000000002,1.25,1.3,1.35,1.4000000000000001,1.4500000000000002,1.5,1.55,1.6,1.6500000000000001,1.7000000000000002,1.75,1.8,1.85,1.9000000000000001,1.9500000000000002,2.0
0,Facebook Spend,4.89,0.101000,0.202000,0.303000,0.40400,0.505000,0.606000,0.707000,0.80800,0.909000,1.010000,1.111000,1.212000,1.313000,1.414000,1.515000,1.616000,1.717000,1.840104,2.781776,3.723449,5.033621,6.343794,7.653966,8.350000,8.350000,8.350000,8.350000,8.350000,8.350000,8.350000,8.350000,8.350000,8.350000,8.350000,8.350000,8.350000,8.350000,8.350000,8.350000,8.350000
1,Instagram Spend,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,G Adwords Spend,0.00,0.012500,0.025000,0.037500,0.05000,0.062500,0.075000,0.087500,0.10000,0.112500,0.125000,0.137500,0.150000,0.162500,0.175000,0.187500,0.200000,0.212500,0.225000,0.237500,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,1.105691,2.415863,3.726036,5.036208,6.346381,7.656553,8.966725,9.110000
3,YouTube Spend,0.00,0.356000,0.712000,1.068000,1.42400,1.780000,2.136000,2.492000,2.84800,3.204000,3.560000,3.916000,4.272000,4.628000,4.984000,5.340000,5.696000,6.052000,6.408000,6.764000,7.120000,7.120000,7.120000,7.120000,7.734139,9.044311,10.354484,11.664656,12.974828,14.285001,15.595173,16.905346,18.215518,18.670000,18.670000,18.670000,18.670000,18.670000,18.670000,18.670000,18.670000
4,Display Spend,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Facebook Collab Spend,9.23,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.004707,0.84538,1.686052,2.526724,3.367397,4.208069,5.048742,5.889414,6.730087,7.570759,8.411432,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000,9.230000
6,Instagram Collab Spend,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,AMS Display Spend,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,AMS Brand Search Spend,1.13,0.702172,1.130000,1.130000,1.13000,1.130000,0.294035,1.130000,1.13000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.130000,1.259138
9,AMS Product Search Spend,4.75,0.138500,0.551345,1.392017,2.23269,3.073362,4.750000,4.750000,4.75000,4.750000,4.750000,4.7500

In [11]:
##### MAX BPM Super Event 
#### do not executer for saffolo oils & oats
#Output_sup_event = pd.DataFrame()
#Output_sup_event = pd.DataFrame(sim_input.FEATURE.unique())
#Output_sup_event.columns = ['FEATURE']
#for i in np.arange(0, 2.01, 0.05):
#    print("############ change ##########")
#    print(i)
#    b= liner_programming_scenario(sim_input, change_value = i, event_tag = 2)
#    col_name = round(i,2)
#    b.rename(columns = {'new_spend': col_name}, inplace = True)
#    b.drop('FEATURE', axis =1, inplace = True)
#    print(b)
#    Output_sup_event= pd.concat([Output_sup_event, b], axis =1)

In [12]:
##### MAX BPM non_event

outputs_non_event = pd.DataFrame(sim_input.FEATURE.unique())
outputs_non_event.columns = ['FEATURE']
for i in np.arange(0, 2.01, 0.05):
    print("############ change ##########")
    print(i)
    b= liner_programming_scenario(sim_input, change_value = i, event_tag=0 )
    col_name = round(i,2)
    b.rename(columns = {'new_spend': col_name}, inplace = True)
    b.drop('FEATURE', axis =1, inplace = True)
    print(b)
    outputs_non_event= pd.concat([outputs_non_event, b], axis =1)

############ change ##########
0.0
Lower range
0
Upper
76.19
#################### Spend data ############################
                     FEATURE   ROI spend_type  upper_range  lower_range
0             Facebook Spend  2.70                   10.60            0
1            Instagram Spend  0.00         ns         0.00            0
2            G Adwords Spend  0.44                    7.20            0
3              YouTube Spend  2.30                   48.20            0
4              Display Spend  0.00         ns         0.00            0
5      Facebook Collab Spend  4.72                    7.00            0
6     Instagram Collab Spend  0.00         ns         0.00            0
7          AMS Display Spend  0.00         ns         0.00            0
8     AMS Brand Search Spend  5.00                    0.42            0
9   AMS Product Search Spend  5.00                    2.77            0
10  AMS Search Display Spend  0.00         ns         0.00            0
Spend Variable

In [13]:
outputs_non_event

,FEATURE,0.0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0,1.05,1.1,1.15,1.2,1.25,1.3,1.35,1.4,1.45,1.5,1.55,1.6,1.65,1.7,1.75,1.8,1.85,1.9,1.95,2.0
0,Facebook Spend,0.0,0.12000,0.240000,0.360000,0.480000,0.600000,0.720000,0.840000,0.960000,1.080000,1.200000,1.320000,1.440000,1.560000,1.680000,1.800000,2.489513,3.281982,4.074452,4.866921,5.659391,7.19186,8.72433,10.256799,10.600000,10.600000,10.600000,10.600000,10.600000,10.600000,10.600000,10.600000,10.600000,10.600000,10.600000,10.600000,10.600000,10.600000,10.600000,10.600000,10.600000
1,Instagram Spend,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,G Adwords Spend,0.0,0.07000,0.140000,0.210000,0.280000,0.350000,0.420000,0.490000,0.560000,0.630000,0.700000,0.770000,0.840000,0.910000,0.980000,1.050000,1.120000,1.190000,1.260000,1.330000,1.400000,1.40000,1.40000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000
3,YouTube Spend,0.0,0.67000,1.340000,2.010000,2.680000,3.350000,4.020000,4.690000,5.360000,6.030000,6.700000,7.370000,8.040000,8.710000,9.380000,10.050000,10.720000,11.390000,12.060000,12.730000,13.400000,13.40000,13.40000,13.400000,14.589269,16.121738,17.654208,19.186677,20.719147,22.251616,23.784086,25.316556,26.849025,28.381495,29.913964,31.446434,32.978903,34.511373,36.043842,37.576312,39.108781
4,Display Spend,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Facebook Collab Spend,0.0,0.00000,0.000000,0.000000,0.000000,0.172348,0.844817,1.517287,2.189756,2.862226,3.534695,4.207165,4.879634,5.552104,6.224573,6.897043,7.000000,7.000000,7.000000,7.000000,7.000000,7.00000,7.00000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000
6,Instagram Collab Spend,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,AMS Display Spend,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,AMS Brand Search Spend,0.0,0.01550,0.031000,0.046500,0.062000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.42000,0.42000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000
9,AMS Product Search Spend,0.0,0.65697,1.313939,1.970909,2.627878,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.77000,2.77000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2.770000,2

In [14]:
writer = pd.ExcelWriter('Simulation_Output_Spend_targeting_Saffola_Oil.xlsx', engine='xlsxwriter')

#store your dataframes in a  dict, where the key is the sheet name you want
frames = {'Event': outputs_event,
          #'Super_Event': Output_sup_event, 
          'Non_Event': outputs_non_event }

#now loop thru and put each on a specific sheet
for sheet, frame in  frames.items(): 
    frame.to_excel(writer, sheet_name = sheet, index = False)

#critical last step
writer.save()

In [15]:
#END#